In [1]:
import sys
import os
print(f"Python Executable: {sys.executable}")
print(f"Current Working Directory: {os.getcwd()}")
try:
    import langchain_openai
    print("✅ langchain_openai is installed!")
except ImportError:
    print("❌ langchain_openai is NOT installed in this environment.")

Python Executable: /Users/mahen/learnings/AI/ai-agent/agentic-ai/langchain/tools/.venv/bin/python
Current Working Directory: /Users/mahen/learnings/AI/ai-agent/agentic-ai/langchain/tools


✅ langchain_openai is installed!


# LangGraph + GPT-4 Hello World

This notebook demonstrates how to build a simple conversational agent using LangGraph and OpenAI's GPT-4.

In [2]:
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langgraph.graph import StateGraph, add_messages

# Load API keys from .env file
load_dotenv()

True

## Define the Agent State

The state uses `Annotated[list[BaseMessage], add_messages]` which means:
- Type: A list of messages
- Reducer: `add_messages` appends new messages instead of replacing them

In [3]:
class AgentState(TypedDict):
    """State schema for our conversational agent."""
    messages: Annotated[list[BaseMessage], add_messages]

## Define the Model Calling Function

This function:
1. Initializes GPT-4
2. Adds a system message for context
3. Invokes GPT-4 with the conversation history
4. Returns the response

In [4]:
def call_model(state: AgentState) -> dict:
    """Invoke GPT-4 with the conversation history."""
    
    # Initialize GPT-4
    llm = ChatOpenAI(
        model="gpt-4",           # Use GPT-4 model
        temperature=0.7,         # Controls randomness (0 = deterministic, 2 = very creative)
        max_tokens=500           # Limit response length
    )
    
    # Add a system message to set the assistant's behavior
    system_msg = SystemMessage(
        content="You are a helpful and friendly AI assistant. Be concise but informative."
    )
    
    # Combine system message with conversation history
    messages = [system_msg] + state["messages"]
    
    # Invoke GPT-4 and get the response
    response = llm.invoke(messages)
    
    # Return the response (it will be appended to messages)
    return {"messages": [response]}

## Build the LangGraph

Create a simple graph with one node that calls GPT-4.

In [5]:
# Create the graph
graph = StateGraph(AgentState)

# Add the model calling node
graph.add_node("assistant", call_model)

# Set entry and finish points
graph.set_entry_point("assistant")
graph.set_finish_point("assistant")

# Compile the graph
app = graph.compile()

print("✅ Graph compiled successfully!")

✅ Graph compiled successfully!


## Test 1: Simple Question

In [6]:
# Test with a simple question
result = app.invoke({
    "messages": [HumanMessage(content="What is LangGraph?")]
})

# Print the conversation
print("=== Conversation ===")
for msg in result["messages"]:
    print(f"\n{msg.type.upper()}: {msg.content}")

=== Conversation ===

HUMAN: What is LangGraph?

AI: LangGraph is not a universally recognized term or tool in technology or linguistics. It might be a specific tool or concept within a certain context, organization, or software. However, without more specific information, it's challenging to provide a detailed explanation. Please provide more context.


## Test 2: Multi-turn Conversation

Demonstrate how messages accumulate in the state.

In [7]:
# Start a multi-turn conversation
conversation = {
    "messages": [
        HumanMessage(content="Hi! My name is Mahendran."),
    ]
}

# First turn
result1 = app.invoke(conversation)
print("=== Turn 1 ===")
print(f"Assistant: {result1['messages'][-1].content}")

# Second turn - the state now contains previous messages
result1["messages"].append(HumanMessage(content="What's my name?"))
result2 = app.invoke(result1)
print("\n=== Turn 2 ===")
print(f"Assistant: {result2['messages'][-1].content}")

# Print full conversation
print("\n=== Full Conversation ===")
for msg in result2["messages"]:
    if msg.type in ["human", "ai"]:
        print(f"\n{msg.type.upper()}: {msg.content}")

=== Turn 1 ===
Assistant: Hello, Mahendran! How can I assist you today?

=== Turn 2 ===
Assistant: Your name is Mahendran.

=== Full Conversation ===

HUMAN: Hi! My name is Mahendran.

AI: Hello, Mahendran! How can I assist you today?

HUMAN: What's my name?

AI: Your name is Mahendran.


## Test 3: Different GPT-4 Models

Try different OpenAI models to see the difference.

In [ ]:
def test_model(model_name: str, question: str):
    """Test a specific OpenAI model."""
    
    def call_specific_model(state: AgentState) -> dict:
        llm = ChatOpenAI(model=model_name, temperature=0)
        system_msg = SystemMessage(content="You are a helpful assistant.")
        messages = [system_msg] + state["messages"]
        response = llm.invoke(messages)
        return {"messages": [response]}
    
    g = StateGraph(AgentState)
    g.add_node("assistant", call_specific_model)
    g.set_entry_point("assistant")
    g.set_finish_point("assistant")
    compiled = g.compile()
    
    result = compiled.invoke({"messages": [HumanMessage(content=question)]})
    return result["messages"][-1].content

# Compare different models
question = "Explain quantum computing in one sentence."
print(f"Question: {question}\n")

# GPT-3.5 Turbo (faster, cheaper)
print("GPT-3.5 Turbo:")
print(test_model("gpt-3.5-turbo", question))

# GPT-4 (more capable, slower)
print("\nGPT-4:")
print(test_model("gpt-4", question))

# GPT-4 Turbo (faster GPT-4)
print("\nGPT-4 Turbo:")
print(test_model("gpt-4-turbo", question))

## 📝 Key Takeaways

1. **`ChatOpenAI(model="gpt-4")`** - Initializes the GPT-4 model
2. **`llm.invoke(messages)`** - Sends messages and gets a response
3. **`add_messages` reducer** - Automatically appends new messages to conversation history
4. **System Message** - Sets the assistant's behavior and context
5. **State Management** - LangGraph handles conversation state automatically

## 🔑 Requirements

Make sure you have a `.env` file with:
```
OPENAI_API_KEY=sk-your-key-here
```

## 📚 Model Options

- `gpt-3.5-turbo` - Fast, cheap, good for simple tasks
- `gpt-4` - Most capable, slower, more expensive
- `gpt-4-turbo` - Fast GPT-4 with larger context window
- `gpt-4o` - Multimodal GPT-4 (text + images)